In [24]:
import json
import time
import datetime
import bs4 as bs4
import requests
import random
import pandas as pd

In [25]:
Web_adress= 'https://www.asianmetal.com/price/productPriceChartEn.am'

##alle seiten aus asian metals
web_list={'https://www.asianmetal.com/price/initPriceListEn.am?pId=40&priceShowAttr=1&paramId=343&isProduct=true':{343:"Magnesium-China",345:"Magnesium-Roterdam"},
          'https://www.asianmetal.com/Ferrochrome/':{274:"Ferrochrome"},
         'https://www.asianmetal.com/price/productPriceChartEn.am?isProduct=true&pId=13&priceParamId=291&strYear=2023&strMonth=11&strDay=9&year=2024&month=2&day=7':{291:"Ferro-molybdenum"},
         'https://www.asianmetal.com/price/priceFactoryChartEn.am?isProduct=true&priceParamId=3905&strYear=2023&strMonth=11&strDay=9&year=2024&month=2&day=7&priceFlag=26&cityId=2601':{3905:"Bearing Round Steel"},
          'https://www.asianmetal.com/price/productPriceChartEn.am?isProduct=true&pId=144&priceParamId=120&strYear=2023&strMonth=11&strDay=9&year=2024&month=2&day=7':{120:"Graphite Electrode"}}

CURRENT_DAY = datetime.datetime.now().day.__str__()
CURRENT_YEAR = datetime.datetime.now().year.__str__()
CURRENT_MOUNTH = datetime.datetime.now().month.__str__()
def parseWebpage(page):
    index=["date","low","high","mid"]
    value_list={}
    soup = bs4.BeautifulSoup(page,features="lxml")
    ma = soup.select_one(selector='#tabledata > tr')
    data= ma.select(selector='td')
    for index_list, i in enumerate(data[0:4]):
        value_list[index[index_list]] = str(i).split("\n")[1].replace("\t","").replace("\r","")
    return value_list
def get_data_id(adress,id):
    headers = {
        'authority': 'www.asianmetal.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
        'cache-control': 'max-age=0',
        'content-type': 'application/x-www-form-urlencoded',
        'cookie': 'JSESSIONID=4ABC3C71E460617A2A3149315BABEA10; YW11=bWJpbWJp',
        'origin': 'https://www.asianmetal.com',
        'referer': str(adress),
        'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Opera GX";v="106"',
        'sec-ch-ua-mobile': '?1',
        'sec-ch-ua-platform': '"Android"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36',
    }

    params = (
        ('priceParamId', str(id)),
        ('strYear', '2023'),
        ('strMonth', '11'),
        ('strDay', '9'),
        ('year', CURRENT_YEAR),
        ('month', CURRENT_MOUNTH),
        ('day', CURRENT_DAY),
        ('priceFlag', ''),
        ('pId', '40'),
    )

    data = {
        'priceParamDiv': '343',
        'priceFlag': '',
        'pId': '40',
        'cityId': '',
        'isProduct': 'true',
        'strYear': '2023',
        'strMonth': '11',
        'strDay': '9',
        'year': '2024',
        'month': '2',
        'day': '7'
    }
    response = requests.post('https://www.asianmetal.com/price/productPriceChartEn.am', headers=headers, params=params, data=data)
    if response.status_code == 200:
        data = response.text
        value_dict = parseWebpage(data)
        if value_dict is not None:
            return value_dict
        else:
            print("Fehler beim Datenempfang")
    else:
        print("Autorisation fehlgeschlagen")
if __name__ =='__main__':
    pars_data_unite={}
    pars_data_unite["UpdatedAt"] = str(datetime.datetime.now())
    for instrument in web_list.keys():
        for i in web_list[instrument].keys():
            loaded_data = get_data_id(instrument,i)
            pars_data_unite[web_list[instrument][i]] = loaded_data
            #json.dump(pars_data_unite[web_list[instrument][i]], open("export_data.json", "w+"), ensure_ascii=False, indent=4)
        time.sleep(random.uniform(1, 2.5))

        with open("export_data.json",'w') as output_input:
            json.dump(pars_data_unite,output_input, ensure_ascii=False, indent=4)

In [28]:
def json_to_excel(json_file, excel_file):
    
    with open(json_file, 'r') as f:
        data = json.load(f)

    df = pd.DataFrame(data)

    df.to_excel(excel_file, index=False)

    print("Daten wurden erfolgreich in Excel exportiert.")

if __name__ == '__main__':
    json_file = "export_data.json"
    excel_file = "export_data.xlsx"
    json_to_excel(json_file, excel_file)

Daten wurden erfolgreich in Excel exportiert.
